In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input, Embedding, LSTM, Dense, Dot, Multiply, Flatten, Activation, Concatenate, GRU,Dropout
from keras.models import Model, Sequential
from keras.layers import Layer
from keras import backend as K
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [2]:
data = pd.read_csv('/content/drive/MyDrive/PKL-JTK-Eben/FIX REC/movie.csv')

In [ ]:
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
data['user'] = data['userId'].astype("category").cat.codes
data['movie'] = data['movieId'].astype("category").cat.codes

num_users = len(data['user'].unique())
num_movies = len(data['movie'].unique())
embedding_size = 10

In [9]:
# train, test = train_test_split(data, test_size=0.2, random_state=42)

In [4]:
user_ids = data['userId'].values
movie_ids = data['movieId'].values
ratings = data['rating'].values

# Create a dictionary to map movie IDs to unique indices
movie_id_to_index = {movie_id: index for index, movie_id in enumerate(np.unique(movie_ids))}

# Map movie IDs and user IDs to their corresponding indices
movie_indices = np.array([movie_id_to_index[movie_id] for movie_id in movie_ids])
user_indices = np.array(user_ids)  # You can similarly create a user index mapping if needed

# Split the data into train and test sets
train_movie_indices, test_movie_indices, train_user_indices, test_user_indices, train_ratings, test_ratings = train_test_split(
    movie_indices, user_indices, ratings, test_size=0.2, random_state=42
)

# Reshape data to match the GRU input shape
train_inputs = np.zeros((len(train_movie_indices), 1, len(np.unique(movie_indices))))
test_inputs = np.zeros((len(test_movie_indices), 1, len(np.unique(movie_indices))))

train_inputs[np.arange(len(train_movie_indices)), 0, train_movie_indices] = 1
test_inputs[np.arange(len(test_movie_indices)), 0, test_movie_indices] = 1

In [8]:
emb_size = 50
hidden_units = 100
train_n_items = len(np.unique(movie_indices))
batch_size = 32
num_epochs = 10


In [21]:
# Define the initial state for the GRU layer
initial_state = K.constant(np.zeros((batch_size, hidden_units)))

inputs = Input(batch_shape=(batch_size, 1, train_n_items))
gru, gru_states = GRU(hidden_units, stateful=True, return_state=True, name="GRU")(inputs, initial_state=[initial_state])
drop2 = Dropout(0.25)(gru)
predictions = Dense(train_n_items, activation='softmax')(drop2)
model = Model(inputs=inputs, outputs=[predictions])


In [22]:
opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False)
model.compile(loss='mean_squared_error', optimizer=opt)


In [23]:
filepath = 'model_checkpoint.h5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=2, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [24]:
# You need to ensure that the number of training examples is divisible by the batch size
num_batches = len(train_inputs) // batch_size
train_inputs = train_inputs[:num_batches * batch_size]
train_ratings = train_ratings[:num_batches * batch_size]


In [26]:
model.fit(train_inputs, train_ratings, batch_size=batch_size, epochs=num_epochs, callbacks=callbacks_list)

Epoch 1/10


TypeError: ignored